In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# Columbia EECS E6893 Big Data Analytics


from pyspark import SparkConf,SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import Row,SQLContext
import datetime
import sys
import requests
import time
import subprocess
import re
from google.cloud import bigquery
from kafka import KafkaProducer

import datetime

def get_reset_seconds():
    date_second = (datetime.datetime.now() + datetime.timedelta(seconds=30)).strftime("%Y-%m-%d") + ' 00:00:00'

    reset_ts = int(datetime.datetime.strptime(date_second, "%Y-%m-%d %H:%M:%S").timestamp())

    return reset_ts - int(datetime.datetime.now().timestamp())

TOPIC = "topic{0}"


# parameter
IP = 'localhost'    # ip port
PORT = 9001     # port

STREAMTIME = 60          # time that the streaming process runs

WORD = ['percent', 'price', 'market', 'sale', 'rise',
        'rate', 'share', 'stock', 'sell', 'increase']     #the words you should filter and do word count



def stock_filter(line):

    # TODO: insert your code here
    

    def isEnglish(s):
        try:
            s.encode(encoding='utf-8').decode('ascii')
        except UnicodeDecodeError:
            return False
        else:
            return True
    
    def containStock(s):
        for word in WORD:
            if word in s:
                return True
        return False
    
    

    line = line.map(lambda x: x.lower()).filter(lambda x: isEnglish(x)).filter(lambda x: containStock(x))
    line = line.map(lambda x: (x, (datetime.datetime.now().timestamp() - start_time.value) // STREAMTIME))
    return line
    

    
def kafka_sink(line, producer):
    producer = producer_sc.value
    def send(tup):
        topic_idx = tup[1]
        msg = tup[0]
        producer.send(TOPIC.format("X"), msg)
        return msg
    
    line = line.map(lambda x: send(x))
    return line
    

def handler(message):
    records = message.collect()
    if len(records) <= 0: return
    idx = int(records[0][1])
    with open("./data/data{0}.txt".format(idx), "a") as f:
        for record in records:
            idx = record[1]
            msg = record[0]
            f.write(msg + "\n")
#         producer.send('topicX', str(msg))
#         producer.flush()


KAFKA_BOOTSTRAP_SERVER = "localhost:9092"

producer = KafkaProducer(bootstrap_servers = KAFKA_BOOTSTRAP_SERVER, 
                                             api_version=(0,11,5))

if __name__ == '__main__':

    # Spark settings
    conf = SparkConf()
    conf.setMaster('local[2]')
    conf.setAppName("TwitterStreamApp")

    sc = SparkContext(conf=conf)
#     sc.setLogLevel("ERROR")
    sc.setLogLevel("OFF")
    start_time = sc.broadcast(datetime.datetime.now().timestamp())
    
#     producer_sc = KafkaProducer(bootstrap_servers = KAFKA_BOOTSTRAP_SERVER, 
#                                              api_version=(0,11,5))
#                                              value_serializer = lambda x: dump(x).encode("utf-8")))
    
# 

    sql_context = SQLContext(sc)
    ssc = StreamingContext(sc, 10)
    # setting a checkpoint to allow RDD recovery
    ssc.checkpoint("~/checkpoint_TwitterApp")

    # read data from port 9001
    dataStream = ssc.socketTextStream(IP, PORT)


    print("---------checkpoint{0}-----------".format(0))



    res = stock_filter(dataStream)
#     res.pprint()
    
    res.foreachRDD(handler)
#     res = kafka_sink(res)
#     producer_sc.send(TOPIC.format("X"), res)
#     res.pprint()

    ssc.start()

    ssc.awaitTerminationOrTimeout(get_reset_seconds()) 
    time.sleep(STREAMTIME)

    ssc.stop(stopSparkContext=False, stopGraceFully=True)

